<a href="https://colab.research.google.com/github/ArtuomMerkulov/Baseline/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Обзор данных

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
!pip list pandas

Package                   Version
------------------------- ------------
addict                    2.4.0
aliyun-python-sdk-core    2.14.0
aliyun-python-sdk-kms     2.16.2
anyio                     4.0.0
appnope                   0.1.3
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     23.1.0
Babel                     2.13.1
beautifulsoup4            4.12.2
bleach                    6.1.0
catboost                  1.2.2
certifi                   2023.7.22
cffi                      1.16.0
charset-normalizer        3.3.0
click                     8.1.7
colorama                  0.4.6
comm                      0.1.4
contourpy                 1.1.1
crcmod                    1.7
cryptography              41.0.4
cycler                    0.12.1
debugpy                   1.8.0
decorator                 5.1.1
defusedxml                0.7.1
EasyProcess        

In [ ]:
data = pd.read_csv("data/transactions.csv")

In [ ]:
data.head()

,client_id,trans_time,mcc_code,trans_type,amount,term_id,trans_city
0,d1bbbc9a0e0410d3cf12a3d2f44f3450,35 08:24:41,4829,2370,-1808.56,NaN,Tver
1,81c7349f7bdcb661bc7d5f5f0fdac6aa,351 13:27:15,6011,7010,7954.51,442207,Kazan
2,fc0c7c937f139bd6d1c5b5d0a0023dd3,150 16:03:59,6011,2010,-12293.56,NaN,Khabarovsk
3,3495487d5beaa5caaa0682fddc38379a,373 11:54:12,6010,7070,49.52,888987,Kaliningrad
4,07e279042b1da23710fa2f65034d30b9,340 13:11:45,6010,7070,45.29,888918,Saint Petersburg


In [ ]:
mcc_data = pd.read_csv("data/mcc_codes.csv", sep=";")

In [ ]:
mcc_data

,mcc_code,mcc_description
0,742,Ветеринарные услуги
1,1711,"Генеральные подрядчики по вентиляции, теплосна..."
2,1731,Подрядчики по электричеству
3,1799,"Подрядчики, специализированная торговля — нигд..."
4,2741,Разнообразные издательства/печатное дело
...,...,...
179,9211,"Судовые выплаты, включая алименты и детскую по..."
180,9222,Штрафы
181,9311,Налоговые платежи
182,9399,"Правительственные услуги, нигде ранее не класс..."


Уникальных пользователей

In [ ]:
len(data.client_id.unique())

8400

Уникальных mmcv

In [ ]:
len(data.mcc_code.unique())

184

### Baseline с прошлого соревнования

In [ ]:
import sys
import os
import warnings
os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import time
import scipy
# import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
!pip install faiss-cpu

In [ ]:
LOCAL_DATA_PATH = ''
SPLIT_SEED = 42

In [ ]:

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3563529 entries, 0 to 3563528
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   client_id   object 
 1   trans_time  object 
 2   mcc_code    int64  
 3   trans_type  int64  
 4   amount      float64
 5   term_id     object 
 6   trans_city  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 190.3+ MB


In [ ]:
round(data.describe(), 3)

,mcc_code,trans_type,amount
count,3563529.000,3563529.000,3563529.000
mean,5586.428,2439.774,-690.021
std,604.201,2215.307,17373.587
min,742.000,1000.000,-4338731.900
25%,5211.000,1030.000,-722.340
50%,5541.000,1110.000,-181.520
75%,6010.000,2370.000,-37.060
max,9402.000,8146.000,4700292.890


In [ ]:
data.groupby("client_id").agg({"amount":"count"})

,amount
client_id,
0002cf30347684df542e1a931f356875,504
0002f4661e0d4d2ae097d78552dad022,267
00081e0eca8ad94e493691f8f00099c6,191
0019bebcb5435df3103a5b0b52f5fc6b,966
001c8191ecac499718b33dc293a99e5b,206
...,...
ffe87ae84cf5d83af6a8a5de6dfe327f,956
fff69a007a119605d6ff25c086840e7d,349
fff78af489ca1024ba0ac7b3c2e47b60,175


In [ ]:
%%time

data_agg = data[['client_id', 'mcc_code', 'amount']].groupby(['client_id', 'mcc_code']).agg({'amount':"count"})

CPU times: user 314 ms, sys: 57.7 ms, total: 371 ms
Wall time: 389 ms


In [ ]:
data_agg

amount
client_id                        mcc_code        
0002cf30347684df542e1a931f356875 4814           9
                                 4829         107
                                 5211           1
                                 5331           5
                                 5399           3
...                                           ...
fffbed6c440a05e5593ad1a865e586a8 6011          44
                                 7523           1
fffedf876a0ea3d39e54b706165a4826 4814           2
                                 5411           1
                                 6011          61

[167998 rows x 1 columns]

### Генерим фичи

In [ ]:
data.groupby("client_id").agg({"trans_city":"first"})

,trans_city
client_id,
0002cf30347684df542e1a931f356875,Saint Petersburg
0002f4661e0d4d2ae097d78552dad022,Novosibirsk
00081e0eca8ad94e493691f8f00099c6,Khabarovsk
0019bebcb5435df3103a5b0b52f5fc6b,Kaliningrad
001c8191ecac499718b33dc293a99e5b,Vladivostok
...,...
ffe87ae84cf5d83af6a8a5de6dfe327f,Novosibirsk
fff69a007a119605d6ff25c086840e7d,Vladivostok
fff78af489ca1024ba0ac7b3c2e47b60,Vladimir


### Продолжаем дальше

In [ ]:
data_agg["client_id"] = [i[0] for i in data_agg.index]
data_agg["mcc_code"] = [i[1] for i in data_agg.index]

In [ ]:
data_agg

amount  \
client_id                        mcc_code           
0002cf30347684df542e1a931f356875 4814           9   
                                 4829         107   
                                 5211           1   
                                 5331           5   
                                 5399           3   
...                                           ...   
fffbed6c440a05e5593ad1a865e586a8 6011          44   
                                 7523           1   
fffedf876a0ea3d39e54b706165a4826 4814           2   
                                 5411           1   
                                 6011          61   

                                                                  client_id  \
client_id                        mcc_code                                     
0002cf30347684df542e1a931f356875 4814      0002cf30347684df542e1a931f356875   
                                 4829      0002cf30347684df542e1a931f356875   
                                 5211      0002cf30347684df542e1a931f356875   
                                 5331      0002cf30347684df542e1a931f356875   
                                 5399      0002cf30347684df542e1a931f356875   
...                                                                     ...   
fffbed6c440a05e5593ad1a865e586a8 6011      fffbed6c440a05e5593ad1a865e586a8   
                                 7523      fffbed6c440a05e5593ad1a865e586a8   
fffedf876a0ea3d39e54b706165a4826 4814      fffedf876a0ea3d39e54b706165a4826   
                                 5411      fffedf876a0ea3d39e54b706165a4826   
                                 6011      fffedf876a0ea3d39e54b706165a4826   

                                           mcc_code  
client_id                        mcc_code            
0002cf30347684df542e1a931f356875 4814          4814  
                                 4829          4829  
                                 5211          5211  
                                 5331          5331  
                                 5399          5399  
...                                             ...  
fffbed6c440a05e5593ad1a865e586a8 6011          6011  
                                 7523          7523  
fffedf876a0ea3d39e54b706165a4826 4814          4814  
                                 5411          5411  
                                 6011          6011  

[167998 rows x 3 columns]

In [ ]:
data_agg = data_agg.reset_index(drop=True)

In [ ]:
mcc_set = set(data_agg['mcc_code'].unique())
print(f'{len(mcc_set)} urls')
url_dict = {mcc: idmcc for mcc, idmcc in zip(mcc_set, range(len(mcc_set)))}
client_set = set(data_agg['client_id'].unique())
print(f'{len(client_set)} users')
usr_dict = {usr: user_id for usr, user_id in zip(client_set, range(len(client_set)))}

184 urls
8400 users


In [ ]:
data_agg.columns

Index(['amount', 'client_id', 'mcc_code'], dtype='object')

In [ ]:
# %%time
values = np.array(data_agg['amount'])
rows = np.array(data_agg['client_id'].map(usr_dict))
cols = np.array(data_agg['mcc_code'].map(url_dict))


mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1))

In [ ]:
data_agg['amount']

0           9
1         107
2           1
3           5
4           3
         ... 
167993     44
167994      1
167995      2
167996      1
167997     61
Name: amount, Length: 167998, dtype: int64

In [ ]:
# u_factors = als.model.item_factors
# d_factors = als.model.user_factors

In [ ]:
# np.dot(d_factors, u_factors.T)

In [ ]:
targets = pd.read_csv("data/train.csv", index_col=0)

In [ ]:
# %%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
usr_emb = pd.DataFrame(mat.toarray()) #usr_emb = pd.DataFrame(u_factors)
usr_emb['client_id'] = usr_emb.index.map(inv_usr_map)
usr_targets = targets
df = usr_emb
## whole_df = data_merged.merge(df, how='inner', on=["user_id"])

# Тут добавляем разные фичи в df

Фичи Артема

In [ ]:
import pandas as pd

def generate_features(dataframe):
  # Total amount per mcc per user
  total_per_mcc = dataframe.groupby(['client_id', 'mcc_code'])['amount'].sum().reset_index()
  total_per_mcc.rename(columns={'amount': 'total_per_mcc'}, inplace=True)

  # Number of transactions per mcc per user
  transactions_per_mcc = dataframe.groupby(['client_id', 'mcc_code'])['amount'].count().reset_index()
  transactions_per_mcc.rename(columns={'amount': 'transactions_per_mcc'}, inplace=True)

  # Join to main dataframe
  data = dataframe.merge(total_per_mcc, how='left', on=['client_id', 'mcc_code'])
  dt = data.merge(transactions_per_mcc, how='left', on=['client_id', 'mcc_code'])

  return dt

def unique_mcc_per_user(dataframe):
  unique_mcc_counts = dataframe.groupby('client_id')['mcc_code'].nunique().reset_index()
  unique_mcc_counts.rename(columns={'mcc_code': 'unique_mcc_count'}, inplace=True)
  return unique_mcc_counts


# Пример использования
features_3 = generate_features(data)
features_unique_mcc = unique_mcc_per_user(data_agg)
features_history_check = mcc_transaction_history(data_agg, 12345, '7523')  # Здесь 12345 - user_id, '7523' - mcc_code

print("## Доп.фичи")
print("Результат на основе генерации фичей:")
display(features_3)
print("Результат по уникальным mcc_codes на пользователя:")
display(features_unique_mcc)

## Доп.фичи
Результат на основе генерации фичей:


,client_id,trans_time,mcc_code,trans_type,amount,term_id,trans_city,total_per_mcc,transactions_per_mcc
0,d1bbbc9a0e0410d3cf12a3d2f44f3450,35 08:24:41,4829,2370,-1808.56,NaN,Tver,-3607723.86,454
1,81c7349f7bdcb661bc7d5f5f0fdac6aa,351 13:27:15,6011,7010,7954.51,442207,Kazan,50009.48,54
2,fc0c7c937f139bd6d1c5b5d0a0023dd3,150 16:03:59,6011,2010,-12293.56,NaN,Khabarovsk,-831011.63,46
3,3495487d5beaa5caaa0682fddc38379a,373 11:54:12,6010,7070,49.52,888987,Kaliningrad,560168.53,356
4,07e279042b1da23710fa2f65034d30b9,340 13:11:45,6010,7070,45.29,888918,Saint Petersburg,5310787.78,10607
...,...,...,...,...,...,...,...,...,...
3563524,e91858185ae43cf3970220748893a01b,328 13:45:38,6010,7070,227.40,888907,Penza,7381156.45,12888
3563525,429b6c8255f13bdb506fdbd6e342ccb1,10 19:22:35,4814,1030,-1084.44,NaN,Khabarovsk,-66615.47,87
3563526,9443b6a0d948cf368d8d72eb295bdfdf,167 23:59:15,5411,1110,-569.20,NaN,Tver,-5669.73,11
3563527,1be75d38ad89c0f2b4d85b95f0fd8069,101 23:59:03,5814,1110,-71.43,NaN,Vladivostok,-3890.25,40


Результат по уникальным mcc_codes на пользователя:


,client_id,unique_mcc_count
0,0002cf30347684df542e1a931f356875,30
1,0002f4661e0d4d2ae097d78552dad022,24
2,00081e0eca8ad94e493691f8f00099c6,11
3,0019bebcb5435df3103a5b0b52f5fc6b,22
4,001c8191ecac499718b33dc293a99e5b,21
...,...,...
8395,ffe87ae84cf5d83af6a8a5de6dfe327f,39
8396,fff69a007a119605d6ff25c086840e7d,37
8397,fff78af489ca1024ba0ac7b3c2e47b60,9
8398,fffbed6c440a05e5593ad1a865e586a8,9


In [ ]:
# generate_features(df)

Фичи из бейзлайна

In [ ]:
_agg_features = ["max", "min", "std", "mean", "count", "median"]
added_features = data.groupby("client_id").agg({"amount":["max", "min", "std", "mean", "count", "median"]})
added_features.columns = _agg_features
added_features = added_features.reset_index(drop=True)
added_features["client_id"] = df["client_id"]
added_features

,max,min,std,mean,count,median,client_id
0,42665.12,-39772.06,7291.653648,-590.822401,504,-220.100,128f3c17a5bc3ff9a60bc00487eeff86
1,15909.56,-9400.52,1213.397340,-199.786367,267,-117.710,c2501d0dda53ff40228cea5b4c62605b
2,5423.80,-5424.42,1196.681764,-70.141309,191,-125.140,13291e10f71543da4aa941b78b6aa2e3
3,20247.50,-36518.54,2109.594352,-189.985704,966,-146.665,bb2e8c3a5afd1798bb092ebc1b182014
4,3616.52,-15907.79,2853.228105,-893.463350,206,-109.965,98b630756cf37b76566aba86a0843fb5
...,...,...,...,...,...,...,...
8395,36157.09,-37944.73,4746.078273,-556.529623,956,-328.225,2c2fac44e3930885fd9894338edffdfc
8396,10847.20,-10756.68,1539.874884,-463.440745,349,-217.550,72109129415454e5d22feccd8526e8f0
8397,7231.79,-36156.56,5207.843362,-1470.981886,175,-145.330,6a718f1d459edfeb504c019d932bb16a
8398,25308.26,-23639.09,4283.052479,442.332845,116,-65.930,9096f2526e11abfc265fd5b645fd4703


In [ ]:
df = df.merge(added_features, how = 'inner', on = ['client_id'])

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,181,182,183,client_id,max,min,std,mean,count,median
0,0.0,0.0,0.00,0.0,0.00,-1668.03,0.0,-3178.21,0.0,0.00,...,0.0,0.0,-1511.19,128f3c17a5bc3ff9a60bc00487eeff86,42665.12,-39772.06,7291.653648,-590.822401,504,-220.100
1,0.0,0.0,-491.45,0.0,0.00,0.00,0.0,0.00,0.0,-2523.83,...,0.0,0.0,-412.56,c2501d0dda53ff40228cea5b4c62605b,15909.56,-9400.52,1213.397340,-199.786367,267,-117.710
2,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00,...,0.0,0.0,0.00,13291e10f71543da4aa941b78b6aa2e3,5423.80,-5424.42,1196.681764,-70.141309,191,-125.140
3,0.0,0.0,0.00,0.0,0.00,0.00,0.0,-5878.28,0.0,-5973.22,...,0.0,0.0,0.00,bb2e8c3a5afd1798bb092ebc1b182014,20247.50,-36518.54,2109.594352,-189.985704,966,-146.665
4,0.0,0.0,-1322.62,0.0,-650.71,-2436.14,0.0,-571.13,0.0,-1801.18,...,0.0,0.0,0.00,98b630756cf37b76566aba86a0843fb5,3616.52,-15907.79,2853.228105,-893.463350,206,-109.965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8395,0.0,0.0,-233.27,0.0,0.00,0.00,0.0,0.00,0.0,0.00,...,0.0,0.0,0.00,2c2fac44e3930885fd9894338edffdfc,36157.09,-37944.73,4746.078273,-556.529623,956,-328.225
8396,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,-1327.15,...,0.0,0.0,0.00,72109129415454e5d22feccd8526e8f0,10847.20,-10756.68,1539.874884,-463.440745,349,-217.550
8397,0.0,0.0,-1136.18,0.0,0.00,-108.75,0.0,-11084.62,0.0,-1457.64,...,0.0,0.0,0.00,6a718f1d459edfeb504c019d932bb16a,7231.79,-36156.56,5207.843362,-1470.981886,175,-145.330
8398,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00,...,0.0,0.0,0.00,9096f2526e11abfc265fd5b645fd4703,25308.26,-23639.09,4283.052479,442.332845,116,-65.930


PCA

In [ ]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=120)
# df_pcad = pca.fit_transform(df.drop(["client_id"], axis=1))
# df_pcad = pd.DataFrame(df_pcad)
# df_pcad["client_id"] = df.client_id
# df_pcad

# Делим на train_test

In [ ]:
id_to_submit = pd.read_csv("data/test.csv", index_col=0)
test_data = id_to_submit.merge(df, how = 'inner', on = ['client_id'])

In [ ]:
test_data

,client_id,0,1,2,3,4,5,6,7,8,...,180,181,182,183,max,min,std,mean,count,median
0,002218c52073faaf23bb26df2cffb138,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,23863.70,-25309.45,2705.938186,-135.180864,880,-180.130
1,0069d4050c7aeb341317e2e125e8fd2e,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,195242.18,-28924.32,11017.206567,-420.442124,339,-722.040
2,0083fb89aea18c408a5534cdd38d7692,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,12366.39,-10774.99,1629.138370,-94.994361,571,-93.090
3,014021fa559786c81e9e1d48d378aabb,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,36156.02,-36156.37,6294.855334,-2326.108541,185,-1443.530
4,01759f92152bbb3de79f9493a4a5cd6d,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,36156.38,-27478.41,2510.330093,-264.305137,586,-158.575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,fe084e295034c4bd6ae4498af6882bb5,0.0,0.0,-672.66,0.0,0.0,0.0,0.0,-2200.99,0.0,...,0.00,0.0,0.0,0.0,5135.15,-9184.26,4458.326118,-4486.182222,27,-6507.390
836,fe9c3e9c58a97ba6c2922237bfc2a7c6,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,88220.94,-36155.83,5896.908755,-372.713788,689,-215.960
837,fee8ac01c79388b623134d61d49c8900,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-542.46,0.0,...,-1084.18,0.0,0.0,0.0,108468.86,-36444.89,3455.884618,6.444849,6919,167.130
838,ff3fe45f7240273e9e22dbffbdf8fd48,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,36156.08,-36156.79,6262.991201,-1651.842735,340,-430.775


In [ ]:
train_val_data = usr_targets.merge(df, how = 'inner', on = ['client_id'])

In [ ]:
train_val_data

,client_id,gender,0,1,2,3,4,5,6,7,...,180,181,182,183,max,min,std,mean,count,median
0,c5f18a1d1d83fda62e51e6ae1d7b8911,1,0.0,0.0,-378.73,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.00,-0.37,-5133.92,1533.124740,-1706.213800,50,-1662.390
1,4b6861803556de90249e13d7cf66f0a9,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-722.84,...,0.00,0.0,0.0,0.00,57849.42,-47363.78,5804.202452,60.352322,887,-241.960
2,7d3f908e1cd1e3624a1445df5e04c4fa,1,0.0,0.0,-1846.52,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.00,36156.89,-180781.28,16154.862756,-4863.830094,744,-990.035
3,40c585a809c1bd17dc6d283d61acd2d2,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-1389.64,...,-650.21,0.0,0.0,-391.03,11570.98,-11496.66,1800.308954,-46.078592,277,-73.940
4,5bd2ff9f6c591353577488b0dfca6dea,0,0.0,0.0,-5458.06,0.0,0.0,0.0,0.0,0.00,...,-1074.63,0.0,0.0,0.00,32540.84,-26754.86,5358.057425,-819.550162,247,-466.780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7555,596baadc284cb94a91c1fdd7a422a022,0,0.0,0.0,-4186.84,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.00,72313.18,-65080.01,10833.628376,-1128.014026,385,-217.290
7556,9b317a4cff6bb3b630cd966f68db410b,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,...,-1807.32,0.0,0.0,0.00,108469.13,-108468.97,13537.803113,-160.818920,324,-377.750
7557,eb3b544e3c5293739f09dcd71a649b18,0,0.0,0.0,-577.20,0.0,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.00,33336.51,-34158.42,3459.102964,-28.848084,694,-173.485
7558,e1c1a70bfcb000d1188745cef71b3b61,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-1299.60,...,0.00,0.0,0.0,0.00,-722.30,-16632.41,3956.496345,-3055.900714,42,-1446.165


# Тестируем

In [ ]:
%%time
x_train, x_val, y_train, y_val = train_test_split(\
    train_val_data.drop(['client_id', 'gender'], axis = 1), train_val_data['gender'], test_size = 0.2, random_state = SPLIT_SEED)
pool_train = Pool(x_train, y_train, cat_features = [])
clf = CatBoostClassifier(depth=5, iterations=900)
pool_val = Pool(x_val, cat_features = [])
clf.fit(pool_train, verbose = False)
print(f'ROC_AUC на тренировочной выборке {m.roc_auc_score(y_train, clf.predict_proba(pool_train)[:,1])}')
print(f'ROC_AUC на валидационной выборке {m.roc_auc_score(y_val, clf.predict_proba(pool_val)[:,1])}')

ROC_AUC на тренировочной выборке 0.9401275614209239
ROC_AUC на валидационной выборке 0.8468911189215265
CPU times: user 20.9 s, sys: 2.38 s, total: 23.3 s
Wall time: 4.39 s


### Подбираем гиперпараметры

In [ ]:
clf.get_all_params()


{'nan_mode': 'Min',
 'eval_metric': 'Logloss',
 'iterations': 1000,
 'sampling_frequency': 'PerTree',
 'leaf_estimation_method': 'Newton',
 'random_score_type': 'NormalWithModelSizeDecrease',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'bayesian_matrix_reg': 0.10000000149011612,
 'eval_fraction': 0,
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': False,
 'model_size_reg': 0.5,
 'pool_metainfo_options': {'tags': {}},
 'subsample': 0.800000011920929,
 'use_best_model': False,
 'class_names': [0, 1],
 'random_seed': 0,
 'depth': 6,
 'posterior_sampling': False,
 'border_count': 254,
 'classes_count': 0,
 'auto_class_weights': 'None',
 'sparse_features_conflict_fraction': 0,
 'leaf_estimation_backtracking': 'AnyImprovement',
 'best_model_min_trees': 1,
 'model_shrink_rate': 0,
 'min_data_in_leaf': 1,
 'lo

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'depth':(3, 5,6, 9, 13), "iterations":(50, 100,300,600, 1000)}
GS = GridSearchCV(clf, parameters)
GS.fit(x_train, y_train, verbose = False)

KeyboardInterrupt: 

In [ ]:
GS.best_params_

{'depth': 3, 'iterations': 50} -> score 0.85 test, 0.89 train

# Формируем сабмит

In [ ]:
%%time
pool_submit = Pool(test_data.drop(["client_id"], axis = 1), cat_features = [])
id_to_submit['gender'] = clf.predict_proba(pool_submit)[:,1]

CPU times: user 23.4 ms, sys: 7.63 ms, total: 31 ms
Wall time: 25.3 ms


,amount
client_id,
0002cf30347684df542e1a931f356875,504
0002f4661e0d4d2ae097d78552dad022,267
00081e0eca8ad94e493691f8f00099c6,191
0019bebcb5435df3103a5b0b52f5fc6b,966
001c8191ecac499718b33dc293a99e5b,206
...,...
ffe87ae84cf5d83af6a8a5de6dfe327f,956
fff69a007a119605d6ff25c086840e7d,349
fff78af489ca1024ba0ac7b3c2e47b60,175


In [ ]:
id_to_submit[(id_to_submit["gender"]>0.4) & (id_to_submit["gender"]<0.6)].merge(data.groupby("client_id").agg({"amount":"count"}), how = 'inner', on = ['client_id'])

,client_id,gender,amount
0,0083fb89aea18c408a5534cdd38d7692,0.473899,355
1,01cdd2938bb35672179f413d33eff98c,0.591552,23
2,0255393cde0896506f6e9f43c13218de,0.470487,904
3,03efb0f0c95daa0b02cabf04679088a4,0.543717,158
4,04f8166edf36837eed65872549e34276,0.416066,61
...,...,...,...
149,f876be13f00417fda4892a53d0d1ac70,0.491106,77
150,f887db4af526b78eee5a1a7ff07609aa,0.598600,774
151,f88a27513bccc68b4f6ee98ace8e8213,0.564413,567
152,fa646b310bbc3837afff4ea1862ea0ec,0.425939,164


In [ ]:
mcc_code = pd.read_csv("data/mcc_codes.csv", sep=";")

In [ ]:
mcc_code

,mcc_code,mcc_description
0,742,Ветеринарные услуги
1,1711,"Генеральные подрядчики по вентиляции, теплосна..."
2,1731,Подрядчики по электричеству
3,1799,"Подрядчики, специализированная торговля — нигд..."
4,2741,Разнообразные издательства/печатное дело
...,...,...
179,9211,"Судовые выплаты, включая алименты и детскую по..."
180,9222,Штрафы
181,9311,Налоговые платежи
182,9399,"Правительственные услуги, нигде ранее не класс..."


In [ ]:
list(data[data.client_id == "0083fb89aea18c408a5534cdd38d7692"].merge(mcc_code, how = 'inner', on = ['mcc_code']).mcc_description)

['Звонки с использованием телефонов, считывающих магнитную ленту',
 'Звонки с использованием телефонов, считывающих магнитную ленту',
 'Звонки с использованием телефонов, считывающих магнитную ленту',
 'Звонки с использованием телефонов, считывающих магнитную ленту',
 'Звонки с использованием телефонов, считывающих магнитную ленту',
 'Звонки с использованием телефонов, считывающих магнитную ленту',
 'Звонки с использованием телефонов, считывающих магнитную ленту',
 'Звонки с использованием телефонов, считывающих магнитную ленту',
 'Звонки с использованием телефонов, считывающих магнитную ленту',
 'Звонки с использованием телефонов, считывающих магнитную ленту',
 'Звонки с использованием телефонов, считывающих магнитную ленту',
 'Звонки с использованием телефонов, считывающих магнитную ленту',
 'Звонки с использованием телефонов, считывающих магнитную ленту',
 'Звонки с использованием телефонов, считывающих магнитную ленту',
 'Звонки с использованием телефонов, считывающих магнитную лен

In [ ]:
id_to_submit

,client_id
0,002218c52073faaf23bb26df2cffb138
1,0069d4050c7aeb341317e2e125e8fd2e
2,0083fb89aea18c408a5534cdd38d7692
3,014021fa559786c81e9e1d48d378aabb
4,01759f92152bbb3de79f9493a4a5cd6d
...,...
835,fe084e295034c4bd6ae4498af6882bb5
836,fe9c3e9c58a97ba6c2922237bfc2a7c6
837,fee8ac01c79388b623134d61d49c8900
838,ff3fe45f7240273e9e22dbffbdf8fd48


## Сабмит

In [ ]:
id_to_submit.head()

,client_id,gender
0,002218c52073faaf23bb26df2cffb138,0.910844
1,0069d4050c7aeb341317e2e125e8fd2e,0.325591
2,0083fb89aea18c408a5534cdd38d7692,0.492943
3,014021fa559786c81e9e1d48d378aabb,0.910688
4,01759f92152bbb3de79f9493a4a5cd6d,0.380092


In [ ]:
id_to_submit.to_csv(f'submission4.csv', index = False)

Объединяем похожие mcc

In [ ]:
{742:1, # ветеринарное
 1711:2, 1731:2, 1799:2, #подрядчики,
 2741:3, #издательства
 3000:4, 4511:4, #авиакомпании, авиалинии
 3351:5, #аренда авто
 3501:6, #жилье отели мотели курорты
 4111:7, 4112:7, 4131:7, #транспортные средства
 4121:8, #лимузины и такси
 4214:9, # грузоперевозки
 4215:10, # услуги курьера
 4411:11, # круизные линии №№№№№№№№№№№№№№№Может добавить в отели мотели курорты
 4722:12, # турагенства
 4784:13, # дорожные пошлины
 4789:14, # транспортировка №№№№№№№№№№№№№№№Может добавить в грузоперевозки
 4812:15, #Телекоммуникации,продажа телефонов
 4814:16, # звонки с магнитных телефонов
 4816:17, # Компьютерная сеть/информационные услуги
 4829:18, # Денежные переводы
 4899:19, # Кабельные и другие платные телевизионные услуги
 4900:20, # Коммуналка
 5013:21, # поставщики
 5039:22, 5211:22, # стройматериалы
 5044:23, # офисное, фото оборудование
 5045:24, 5734:24, # Компы, по, периферия
 5047:0, # оборудование для стоматологов, лабораторий
 5065:25, 5072:25, 5074:25, 5085:25,#оборудование
 5094:26, # драгоценности
 5099:27, # товары длительного пользования
 5111:28, 5943:28, # канцелярия
 5122:29, # лекарства
 5131:30, # Штучные товары, галантерея и другие текстильные товары
 5137:31, # Мужская, женская и детская спец-одежда
 5169:32, # химикалии
 5172:33, # нефть
 5192:34, 5942:34, # книги, издания, газеты
 5193:35, # цветочное оборудование
 5199:36, #Товары недлительного пользования
 5200:40, #Товары для дома
 5231:41, 5713:41, #стекло, краски обои
 5251:42, #скобяные товары
 5261:43, # садовые товары
 5300:44, # оптовики
 5309:45, #duty free
 5310:46, #Магазины, торгующие по сниженным ценам
 5311:47, 5331:47, #Универмаги, универсамы
 5399:48, # товары общего назвначения
 5411:49, # бакалея, супермаркеты
 5422:50, # мясо
 5441:51, # кондитерские
 5451:52, # молочка
 5462:53, # булочные
 5499:54, # продовольственные магазины
 5511:55, # продажа сервис ремонт транспорта
 5532:56, 5533:57, # автозапчасти
 5541:58, #СТО
 5542:59, #Заправки
 5571:60, 5599:60, #мотоциклы снегоходы
 5611:61, # Мужская одежда
 5621:62, # женская одежда
 5631:63, # женские аксессуары
 5641:64, # детская одежда
 5651:65, # одежда для всей семьи
 5655:66, # спортивная одежда для верх езды и мотоцикла
 5661:67, # обувные
 5681:68, # емховые изделия
 5691:69, # Магазины мужской и женской одежды
 5697:70, # портной
 5699:71, # различная одежда и аксессуары
 5712:72, 5722:73, # оборудование мебель быт
 5714:73, #Ткани, обивочный материал, гардины и портьеры, жалюзи
 5719:74, # Различные специализированные магазины бытовых принадлежностей
 5732:76, # Продажа электронного оборудования
 5733:77, # Продажа музыкальных инструментов, фортепиано, нот
 5735:78, # Магазины звукозаписи
 5811:79, # Gоставщики провизии
 5812:80, 5814:80, # общественное питание, рестораны
 5813:81, # бары алко
 5816:82, # Цифровые товары, игры
 5912:83, # Аптеки
 5921:84, # спирт навынос
 5931:85, # second Hand
 5940:86, # веломагазины
 5941:87, # Магазины спорттоваров
 5944:88, # часы, ювелирка



}